In [1]:
%load_ext autoreload
%autoreload 2

import time
import sys
sys.path.append('../')
import cf_ec2.dataPrep as dataPrep

## step 1: import train data

In [2]:
import pandas as pd

data = pd.read_csv('../../rec_model_cf_20190702/metadata/trainData/purchase_20180731_20190725.csv',sep=',',header=0)
data.head(2)

,contact_uid,event_dt,make,model,jd_cat,jd_sub_cat,seller_cat,auction_cd,channel,age,mile,cr_grade,distance_miles,mmr
0,100118431,2019-04-09,FORD,F150 FFV,PICKUP,LITE DUTY FULLS,Retail Dealership,FAAO,Simulcast,5,70737.0,4.3,680.0,15750.0
1,101224608,2018-09-25,NISSAN,ALTIMA,MIDSIZE CAR,PREMIUM,Captive Finance,FAAO,Simulcast,3,36019.0,4.3,2127.0,12000.0


In [3]:
data['ymm'] = [
    '{year}-{make}-{model}'.format(
        year=year,
        make=make,
        model=model
    )
    for year,make,model in zip(data.age,data.make,data.model)
]

train = data[data.event_dt>='2019-04-26'].groupby(['contact_uid','ymm']).size().reset_index().rename(columns={0:'freq'})
test = data[
    (data.event_dt>='2019-03-26')&(data.event_dt<='2019-04-25')
].groupby(['contact_uid','ymm']).size().reset_index().rename(columns={0:'freq'})
train.head(5)

,contact_uid,ymm,freq
0,100000012,10-BUICK-LACROSSE,1
1,100000012,10-CHEVROLET-IMPALA FFV,1
2,100000012,10-CHEVROLET-MALIBU,1
3,100000012,10-CHEVROLET-MALIBU FFV,1
4,100000012,10-FORD-ESCAPE,1


## step 2: test the dataPrep module

In [4]:
dataset = dataPrep.Data(
    train=train,
    test=test,
    col_user='contact_uid',
    col_item='ymm',
    col_rating='freq'
)

In [5]:
dataset.train.head(3)

,contact_uid,ymm,freq
0,0,0,1
1,0,1,1
2,0,2,1


In [6]:
dataset.id2user[0], dataset.id2item[0]

(100000012, '10-BUICK-LACROSSE')

In [7]:
train.head(3)

,contact_uid,ymm,freq,contact_uid_idx,ymm_idx
0,100000012,10-BUICK-LACROSSE,1,0,0
1,100000012,10-CHEVROLET-IMPALA FFV,1,0,1
2,100000012,10-CHEVROLET-MALIBU,1,0,2


In [8]:
t1 = time.time()
dataset.prepTrainDNN()
print('Finished in {} seconds'.format(time.time()-t1))

Finished in 43.76504111289978 seconds


In [9]:
t1 = time.time()
dataset.prepTestDNN()
print('Finished in {} seconds'.format(time.time()-t1))

Finished in 512.3596007823944 seconds


In [11]:
dataset.users[:6],dataset.items[:6],dataset.ratings[:6]

(array([0, 0, 0, 0, 0, 0]),
 array([0, 1, 2, 3, 4, 5]),
 array([1., 1., 1., 1., 1., 3.]))

In [16]:
dataset.test_users[:6],dataset.test_items[:6],dataset.test_ratings[:6]

([0, 0, 0, 0, 0, 0],
 [4345, 5112, 5723, 3590, 7862, 1227],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [20]:
t1 = time.time()
dataset.negativeSampling()
print('Finished in {} seconds'.format(time.time()-t1))

Finished in 222.29684114456177 seconds


In [21]:
dataset.users[:6],dataset.items[:6],dataset.ratings[:6]

(array([0, 0, 0, 0, 0, 0]),
 array([   0, 5104, 5417, 4340, 4935,    1]),
 array([1., 0., 0., 0., 0., 1.]))